In [ ]:
import sys, os
import pandas as pd
sys.path.append('../src/')
from Biologging_Toolkit.processing.Depth import Depth
from Biologging_Toolkit.applications.Drift_Dives import DriftDives

In [ ]:
depid = 'ml17_280a'
path = '/run/media/grosmaan/LaCie/individus_brut/'
sens_path = os.path.join(path, 'CTD', depid, depid+'sens5.nc')
raw_path = os.path.join(path, 'individus', depid, 'raw')

### Increment dives

In [ ]:
inst = Depth(depid, 
            path = os.path.join(path, 'individus',depid), 
            raw_path = raw_path,
            sens_path = sens_path
           )

#### Run inst.get_depth() if depth is not yet in data structure

In [ ]:
inst.get_depth()

In [ ]:
inst()

In [ ]:
inst.ds

In [ ]:
inst.ds.close()